In [ ]:
import pandas as pd
import os
import requests
import time
from tqdm.auto import tqdm
from bs4 import BeautifulSoup


keywords = pd.read_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\data\keywords.csv")

In [ ]:


url = "https://hokify.com/website-api/relations/list"
headers = {
    "Accept": "application/json, text/plain, */*",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en-GB;q=0.9,en;q=0.8",
    "Content-Length": "104",
    "Content-Type": "application/json",
    "Origin": "https://hokify.de",
    "Referer": "https://hokify.de/",
    "Sec-Ch-Ua": '"Not.A/Brand";v="8", "Chromium";v="114", "Google Chrome";v="114"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Ch-Ua-Platform": '"Windows"',
    "Sec-Fetch-Dest": "empty",
    "Sec-Fetch-Mode": "cors",
    "Sec-Fetch-Site": "cross-site",
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
    "X-Top-Level-Domain": "de"
}

data = {"branch":"{keyword}","city":"","filters":[],"isCompanyPage":False,"radius":0,"page":"{pagination}","experimental":None}

df = pd.DataFrame()
df["keyword"] = keywords["general"].dropna().tolist()
df["category"] = "general"
df["url"] =  url
df["headers"] = str(headers)
df["data"] = str(data)
df

In [ ]:

df["data"] = df.apply(lambda row: row["data"].replace("{keyword}", row["keyword"]), axis=1)
df
   


In [9]:
def pagination(row):
    try:
        time.sleep(2)
        url=row["url"]
        data = eval(row["data"].replace("{pagination}", '2'))
        headers = eval(row["headers"])
        response  = requests.post(url, headers=headers, json=data, timeout=5)
        end = response.json()["relationCount"]
        int_end = round(int(end)/15)
        return [i for i in range(1, int_end)]
    except:
        return None

try:
    tqdm.pandas()
    df["pagination"] = df.progress_apply(pagination, axis=1)
    list_columns = df.columns[df.applymap(lambda x: isinstance(x, list)).any()].tolist()
    df = df[df["pagination"].notna()]
    df = df.explode(list_columns)
finally:
    df.to_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\url\hokify.csv", index=False)

In [ ]:
def replace_pagination(row):
       return row["data"].replace("{pagination}", str(row["pagination"]))
         
 


df = pd.read_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\url\hokify.csv")
df = df[df["pagination"].notna()]
df["pagination"] = df["pagination"].astype(int)
df["data"] = df.apply(replace_pagination, axis=1)
df.to_csv(r"C:\Users\besff\OneDrive\Documents\GitHub\datagenerator\datagenerator\url\hokify1.csv", index=False)